In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from hydra import compose, initialize
from omegaconf import OmegaConf

from utils.metric import score

## 提出ファイルを使ってゼロ埋めする

In [3]:
def hide_submission(sub_df):
    hide_sub_df = sub_df.clone().with_columns(
        [pl.lit(0.0).alias(f"ptend_q0002_{i}") for i in range(12, 22)]
    )
    return hide_sub_df

In [4]:
base_sub_path = "output/20240610_ensemble_nelder_mead2_lb078544.parquet"
base_sub_df = pl.read_parquet(base_sub_path)

In [5]:
base_sub_df.head()

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

In [6]:
# hide
hide_base_sub_df = hide_submission(base_sub_df)
print(hide_base_sub_df.shape)
hide_base_sub_df[:, 120:150].head()

(625000, 369)


ptend_q0001_59,ptend_q0002_0,ptend_q0002_1,ptend_q0002_2,ptend_q0002_3,ptend_q0002_4,ptend_q0002_5,ptend_q0002_6,ptend_q0002_7,ptend_q0002_8,ptend_q0002_9,ptend_q0002_10,ptend_q0002_11,ptend_q0002_12,ptend_q0002_13,ptend_q0002_14,ptend_q0002_15,ptend_q0002_16,ptend_q0002_17,ptend_q0002_18,ptend_q0002_19,ptend_q0002_20,ptend_q0002_21,ptend_q0002_22,ptend_q0002_23,ptend_q0002_24,ptend_q0002_25,ptend_q0002_26,ptend_q0002_27,ptend_q0002_28
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.6495e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5384e-52,-1.1441e-50,-3.8119e-46,0.0,0.0,-3.1695e-14
2.8797e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.7835e-38,-1.1512e-34,0.0,-1.5774e-13
-1.6525e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.6324e-55,-4.9586e-50,-1.7488e-45,-3.0572e-41,-1.6994e-36,-4.7132e-32,-5.3210e-14
-7.1831e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.2260e-33,-1.4761e-28,1.1855e-13
-6.1449e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3233e-46,-1.0059e-41,-4.6097e-37,-1.4812e-32,-4.3854e-28,-1.1530e-23,-3.5388e-14


In [7]:
hide_base_sub_df.write_parquet("output/hide_lb078544_sub.parquet")

### location使った予測で20%を埋める

In [8]:
# multi-column to single-column の読み込み
multi_col_sub_path = "output/20240610_ensemble_nelder_mead2_lb080254.parquet"
multi_col_sub_df = pl.read_parquet(multi_col_sub_path)

# 重みが変わったので新しいスケールに変換する
ss_old_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission_old.csv", n_rows=1
)
weight_old_array = ss_old_df.select(
    [x for x in ss_old_df.columns if x != "sample_id"]
).to_numpy()[0]

ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]


multi_col_pred = multi_col_sub_df[:, 1:].to_numpy()
multi_col_pred = multi_col_pred / weight_old_array
multi_col_pred[np.isnan(multi_col_pred)] = 0
multi_col_pred *= weight_array

multi_col_sub_df = pl.concat(
    [
        multi_col_sub_df.select("sample_id"),
        pl.from_numpy(multi_col_pred, schema=multi_col_sub_df.columns[1:]),
    ],
    how="horizontal",
)
multi_col_sub_df

/tmp/ipykernel_2957/4051025496.py:23: RuntimeWarning: invalid value encountered in divide
  multi_col_pred = multi_col_pred / weight_old_array


sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_169651""",-0.000011,-0.000026,-0.000022,-0.000046,-0.000081,-0.000108,-0.000104,-0.000083,-0.000054,-0.000041,-0.000029,-0.000022,-0.000016,-0.00001,-0.000002,0.000002,0.000003,8.2796e-7,0.000004,-9.3783e-7,-0.000003,-0.000006,-0.00001,-0.000013,-0.000014,-0.000016,-0.000019,-0.000024,-0.000026,-0.000032,-0.000034,-0.000032,-0.000031,-0.000028,-0.000027,-0.000028,…,1.3003e-8,-7.8969e-9,6.6855e-8,-7.0226e-7,4.1865e-7,-2.3593e-8,3.6046e-8,3.6364e-8,-5.9829e-8,1.6644e-7,-3.4152e-7,2.3089e-7,-3.7505e-7,6.0852e-7,-1.1834e-7,2.1883e-8,6.0628e-8,4.0804e-8,-7.2421e-9,-8.3721e-9,9.4521e-8,1.1748e-7,1.1309e-7,1.4100e-7,-6.9411e-7,2.4739e-7,-3.7350e-7,-0.000006,0.000007,-0.567619,400.672139,1.0646e-11,2.7601e-10,-0.251745,-0.069717,0.011733,-0.0525
"""test_524862""",-0.000008,-0.000021,-0.000023,-0.000047,-0.000086,-0.000113,-0.000104,-0.00008,-0.000052,-0.000041,-0.000031,-0.000024,-0.000018,-0.000013,-0.000006,-0.000001,8.4510e-7,5.6776e-7,-9.8362e-7,-0.000001,-0.000004,-0.000006,-0.000008,-0.000009,-0.00001,-0.000013,-0.000018,-0.000024,-0.000026,-0.000025,-0.000024,-0.000025,-0.000026,-0.000027,-0.000027,-0.000026,…,-1.9820e-7,-1.1089e-7,3.7925e-8,1.6880e-7,2.4753e-7,3.0584e-7,2.4084e-7,6.8354e-9,-1.8028e-7,-2.6124e-7,-1.2527e-7,-7.8038e-8,-2.8370e-7,0.000001,-2.3399e-8,-1.0843e-7,-3.8337e-7,-4.5829e-7,-3.4414e-7,-5.5615e-7,-4.2788e-7,2.4215e-7,-1.7502e-7,-6.0999e-7,-4.9460e-7,-0.000003,-0.000005,-0.000011,0.000023,-0.069463,388.152037,-7.1783e-12,1.1436e-8,0.205497,0.290388,-0.06125,-0.128024
"""test_634129""",-0.00001,-0.000049,-0.000034,-0.000048,-0.000075,-0.000104,-0.000106,-0.000083,-0.000049,-0.000036,-0.000025,-0.000018,-0.000012,-0.000008,-0.000003,0.000004,0.000006,0.000004,0.000002,-0.000003,-0.000001,-0.000005,-0.000009,-0.000019,-0.000027,-0.000036,-0.000046,-0.000046,-0.000042,-0.000031,-0.000037,-0.000037,-0.000034,-0.000027,-0.000025,-0.000027,…,-0.000002,4.8660e-7,-2.8589e-7,-0.000001,-9.9425e-7,-3.1452e-7,1.2238e-7,2.6674e-7,2.3976e-7,-6.9269e-8,-1.8579e-7,-2.3828e-7,-7.0338e-8,1.1333e-7,-1.4694e-7,-7.2515e-7,-1.3490e-7,2.4356e-9,-3.2708e-7,-6.1495e-7,1.5782e-7,2.6185e-7,6.0001e-7,4.1940e-7,0.000001,0.000001,-3.5972e-7,4.9759e-7,-0.000002,0.107786,425.75918,-3.1899e-11,5.7591e-10,0.28341,0.404113,-0.111441,-0.221363
"""test_403572""",-0.000012,-0.000039,-0.00003,-0.000044,-0.000075,-0.000109,-0.000109,-0.000083,-0.000049,-0.000037,-0.000027,-0.000019,-0.000013,-0.000009,-0.000004,0.000001,0.000005,0.000003,0.000003,-0.000002,-5.7069e-7,-0.000005,-0.00001,-0.000022,-0.000029,-0.000024,-0.00002,-0.000026,-0.000035,-0.000031,-0.000029,-0.000026,-0.000027,-0.000033,-0.000032,-0.000035,…,-1.9160e-7,-5.7978e-7,-2.0680e-7,2.3468e-8,4.5345e-7,4.4238e-7,3.7133e-7,7.2173e-8,-2.1482e-7,-3.4356e-7,-7.6084e-7,-0.000003,7.7067e-7,0.000002,1.0058e-7,1.8989e-7,-2.3146e-7,-6.6126e-7,-0.000001,-0.000003,-0.000004,-0.00000

In [10]:
# 重複範囲を取得
test_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test.csv")
test_old_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test_old.csv")
tmp = test_df[:, 1::10].with_row_index("index")
tmp_old = test_old_df[:, 1::10].with_row_index("old_index")
merge_df = pd.merge(tmp.to_pandas().round(5), tmp_old.to_pandas().round(5))

# 新しいテストでの重複 index
duplicated_index = merge_df["index"]

# 古いテストでの重複 index
duplicated_old_index = merge_df["old_index"]

In [11]:
# 予測値を置き換える

sub_pred = base_sub_df[:, 1:].to_numpy()
multi_col_sub_pred = multi_col_sub_df[:, 1:].to_numpy()
sub_pred[duplicated_index, :] = multi_col_sub_pred[duplicated_old_index, :]


fill_sub_df = pl.concat(
    [
        test_df.select("sample_id"),
        pl.from_numpy(sub_pred, schema=base_sub_df.columns[1:]),
    ],
    how="horizontal",
)
fill_sub_df.head(10)

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

In [12]:
# 最終的に提出したいもの
fill_sub_df.write_parquet("output/fill_lb078544_fill_lb080254_sub.parquet")

In [13]:
# hide
hide_fill_sub_df = hide_submission(fill_sub_df)
print(hide_fill_sub_df.shape)
hide_fill_sub_df[:, 120:150].head()

(625000, 369)


ptend_q0001_59,ptend_q0002_0,ptend_q0002_1,ptend_q0002_2,ptend_q0002_3,ptend_q0002_4,ptend_q0002_5,ptend_q0002_6,ptend_q0002_7,ptend_q0002_8,ptend_q0002_9,ptend_q0002_10,ptend_q0002_11,ptend_q0002_12,ptend_q0002_13,ptend_q0002_14,ptend_q0002_15,ptend_q0002_16,ptend_q0002_17,ptend_q0002_18,ptend_q0002_19,ptend_q0002_20,ptend_q0002_21,ptend_q0002_22,ptend_q0002_23,ptend_q0002_24,ptend_q0002_25,ptend_q0002_26,ptend_q0002_27,ptend_q0002_28
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.6495e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5384e-52,-1.1441e-50,-3.8119e-46,0.0,0.0,-3.1695e-14
2.8797e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.7835e-38,-1.1512e-34,0.0,-1.5774e-13
-1.6525e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.6324e-55,-4.9586e-50,-1.7488e-45,-3.0572e-41,-1.6994e-36,-4.7132e-32,-5.3210e-14
-7.1831e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.2260e-33,-1.4761e-28,1.1855e-13
-6.1449e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3233e-46,-1.0059e-41,-4.6097e-37,-1.4812e-32,-4.3854e-28,-1.1530e-23,-3.5388e-14


In [14]:
hide_fill_sub_df.write_parquet("output/hide_lb078544_fill_lb080254_sub.parquet")

In [16]:
# 埋める前後でスコアが大きく変わらないかをチェック
preds = hide_base_sub_df[:, 1:].to_numpy()
labels = hide_fill_sub_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.9958345362558741


## validationから定数埋めでどれだけスコアが下がりそうか確認

In [ ]:
config_dir = "../experiments"
exp_name = "201_unet_multi/all"

In [4]:
with initialize(
    version_base=None, config_path=f"{config_dir}/{exp_name.split('/')[0]}"
):
    cfg = compose(
        config_name="config.yaml",
        overrides=[f"exp={exp_name.split('/')[-1]}"],
        return_hydra_config=True,
    )

# 定数定義
output_dir = Path(f"output/experiments/{exp_name}")
gcs_path = f"gs://{cfg.dir.gcs_bucket}/{cfg.dir.gcs_base_dir}/experiments/{exp_name}/"

# 結果などの読み込み
r2_score_dict = pickle.load(open(output_dir / "val2_r2_score_dict.pkl", "rb"))
print("r2: ", np.mean(list(r2_score_dict.values())))

/kaggle/working
r2:  0.7815333871933391


In [5]:
predict_df = pl.read_parquet(gcs_path + "val2_predict.parquet", retries=5)
label_df = pl.read_parquet(gcs_path + "val2_label.parquet", retries=5)
ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]

In [41]:
# 通常
preds = predict_df[:, 1:].to_numpy()
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7815333871933391


In [42]:
r2_scores[132:142]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
# 10個の後処理結果を0埋めに変換

preds = predict_df[:, 1:].to_numpy()
preds[:, 132:142] = 0
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7538482690839238


In [47]:
(1 - r2_scores[132:142]).sum() / 368

0.027685118109415333

In [48]:
(1 - r2_scores[132:142]).sum() / 368 + 0.7538482690839238

0.7815333871933391